In [1]:
#Import necessary libraries

import numpy as np
import pandas as pd
import seal
from seal import *
import time
import pickle

In [2]:
#Client reloads encryption context

parms = EncryptionParameters(scheme_type.ckks)



parms.load('encr_parms')



context = SEALContext(parms)

ckks_encoder = CKKSEncoder(context)
slot_count = ckks_encoder.slot_count()

scale = pickle.load(open('scale','rb'))

#Load secret key and initialize decryptor
keygen = KeyGenerator(context)
public_key = keygen.create_public_key()
public_key.load(context, 'pubkey')

secret_key = keygen.secret_key()
secret_key.load(context, 'secretkey')

encryptor = Encryptor(context, public_key)
decryptor = Decryptor(context,secret_key)


In [3]:
num_test_samples = 300

In [4]:
#Load encrypted results
start = time.time()

results = []
for m in range(num_test_samples):
    pt_init = ckks_encoder.encode(0.,scale)
    ct_init = encryptor.encrypt(pt_init)
    ct_init.load(context, 'result[%i]' % m)
    results.append(ct_init)

end = time.time()
print(f"Time to load encrypted results: {(end-start):.3f}s")

Time to load encrypted results: 2.126s


In [5]:
#Decrypt results

ans = []
for i in range(num_test_samples):
    ans.append(decryptor.decrypt(results[i]))
    ans[i] = ckks_encoder.decode(ans[i])

In [6]:
#Form predictions.
#Includes sole post-processing scaling operation

predictions = np.zeros((num_test_samples,5))

for i in range(num_test_samples):
    for j in range(5):
        predictions[i,j] = ans[i][j]/10**3

In [7]:
#Load true phenotype values

ph1_test = pickle.load(open('ph1_test','rb'))
ph2_test = pickle.load(open('ph2_test','rb'))
ph3_test = pickle.load(open('ph3_test','rb'))
ph4_test = pickle.load(open('ph4_test','rb'))
ph5_test = pickle.load(open('ph5_test','rb'))

In [8]:
#Compute ranges for normalization

ph1_range = ph1_test.max()-ph1_test.min()
ph2_range = ph2_test.max()-ph2_test.min()
ph3_range = ph3_test.max()-ph3_test.min()

In [9]:
from sklearn.metrics import mean_squared_error, roc_auc_score

In [10]:
#Normalized scores

print('PH1 NRMSE:', np.sqrt(mean_squared_error(ph1_test, predictions[:,0]))/ph1_range)
print('PH2 NRMSE:', np.sqrt(mean_squared_error(ph2_test, predictions[:,1]))/ph2_range)
print('PH3 NRMSE:', np.sqrt(mean_squared_error(ph3_test, predictions[:,2]))/ph3_range)
print('PH4 ROC: ', roc_auc_score(ph4_test,predictions[:,3]))
print('PH5 ROC: ', roc_auc_score(ph5_test,predictions[:,4]))

PH1 NRMSE: 0.041822228925351757
PH2 NRMSE: 0.030976525958347063
PH3 NRMSE: 0.028704621329617593
PH4 ROC:  0.9859422492401216
PH5 ROC:  0.9811766804942237
